In [2]:
from _storage.list_blobs import list_blobs
from _auth.get_project_id import get_project_id
from _auth.google_auth import google_auth

google_auth()

list_blobs(get_project_id())

['spotify/audio_features/37i9dQZEVXbMDoHDwVN2tF/2022-11-20.csv']

In [5]:
import pandas as pd
from _auth.get_project_id import get_project_id

df = pd.read_csv(f'gs://{get_project_id()}/spotify/audio_features/37i9dQZEVXbMDoHDwVN2tF/2022-11-20.csv')

In [47]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from _spotify import spotipy_auth

spotipy_auth()

sp = Spotify(auth_manager = SpotifyClientCredentials())

res = sp.search('bon iver', type = 'artist')
some_id = res['artists']['items'][0]['id']
artist = sp.artist(some_id)

In [45]:
def _standardise(audio_features: pd.DataFrame, reference: pd.DataFrame) -> pd.DataFrame:
    metrics = [
        "danceability",
        "energy",
        "loudness",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "tempo",
        "duration_ms",
    ]

    for m in metrics:
        audio_features[m] = (
            audio_features[m] - reference.loc[m, "mean"]
        ) / reference.loc[m, "std"]

    return audio_features[metrics].T.to_dict()[0]

def standardised_audio_features(track_id: str) -> dict:
    # Get Reference
    global_top = pd.read_csv(
        "gs://yo-personal-project/spotify/audio_features/37i9dQZEVXbMDoHDwVN2tF/2022-11-20.csv"
    ).set_index("feature")

    # Get audio features from track_id
    sp = Spotify(auth_manager=SpotifyClientCredentials())
    audio_features = pd.DataFrame(sp.audio_features(tracks=track_id))

    # Standardise audio features by global 50
    return _standardise(audio_features, global_top)

In [49]:
std_features = standardised_audio_features('4pvb0WLRcMtbPGmtejJJ6y')

In [52]:
label = list(std_features.keys())

In [56]:
list(std_features.values())

[-0.36892250389953235,
 -0.3632718563510213,
 -0.47833567115794223,
 -0.3570023751225863,
 -0.3256628563782553,
 -0.3551567740036169,
 -0.3583339938144173,
 -0.3673730589309246,
 -0.5278964160636473,
 1.3989850103432933]

In [40]:
sp.artist_top_tracks(some_id)['tracks'][0]['id']

'4pvb0WLRcMtbPGmtejJJ6y'